In [ ]:
%load_ext autoreload
%autoreload 2
import json
import os
from pathlib import Path
from typing import Optional
# from edison_client import EdisonClient, JobNames
from dotenv import load_dotenv

load_dotenv()
# trajectory_id = "2e8d7bbb-8cb9-4b7d-b94a-2e34c8254c59"
# api_key = os.environ.get("EDISON_API_KEY")

# client = EdisonClient(api_key=api_key)
# task_response = client.get_task(trajectory_id)
# task_response

In [ ]:
projects_dir = Path("projects")
project_name = "cpas"
projects_dir.mkdir(exist_ok=True, parents=True)
project_dir = projects_dir / project_name
project_dir.mkdir(exist_ok=True, parents=True)
# with open(project_dir / f"task_response_{trajectory_id}.json", "w") as f:
#     f.write(task_response.model_dump_json(indent=2))

In [ ]:
from papers2dataset.openalex_client import fetch_work, search_works, fetch_work
from papers2dataset.bfs_queue import BFSQueue

q = BFSQueue(project_dir / "bfs_queue.json")
# search_result = search_works("High-Throughput Evaluation of Cryoprotective Agents for")
# q.add_many([x.get("id", "").split('/')[-1] for x in search_result['results']])

In [ ]:
from papers2dataset.openalex_client import fetch_work
pid = q.pop()
paper = fetch_work(pid)

In [ ]:
from papers2dataset.openalex_client import fetch_pdf
pdf_path = fetch_pdf(paper, project_dir)
if pdf_path is None:
    print(f"Failed to download PDF for {paper['id']}")
    q.mark_failed(pid, "no_pdf")

In [ ]:
from papers2dataset.extractor import check_paper_relevance
res = check_paper_relevance(paper)
if not res['has_cpa_compositions']:
    q.mark_skipped(pid, res['reason'])

In [ ]:
res

In [ ]:
from papers2dataset.extractor import extract_cpa_from_pdf
resp = extract_cpa_from_pdf(pdf_path, project_dir)

In [ ]:
response=resp
content = response.choices[0].message.content
result = json.loads(content)

In [ ]:
from papers2dataset.openalex_client import fetch_related_works, fetch_cited_works, fetch_citing_works
related_works = fetch_related_works(pid)
cited_works = fetch_cited_works(pid)
citing_works = fetch_citing_works(pid)

q.add_many([x.get("id", "").split('/')[-1] for x in related_works+cited_works+citing_works])
q.mark_processed(pid)

In [ ]:
pid=q.pop()

In [ ]:
paper = fetch_work(pid)